# Introduction

## Use cql2-json filters to interact with the AERONET API 

See https://aeronet.gsfc.nasa.gov/print_web_data_help_v3.html for details

Decisions:

- AVG - Data Format, All points: AVG=10, use `[{"property": "data_format"}, "all-points"]`
- AVG - Data Format, Daily average: AVG=20, use `[{"property": "data_format"}, "daily-average"]`



In [1]:
cql2_filter = {
    "op": "and",
    "args": [
        {"op": "eq", "args": [{"property": "site"}, "Cart_Site"]},
        {"op": "eq", "args": [{"property": "data_type"}, "AOD10"]},
        {"op": "eq", "args": [{"property": "format"}, "csv"]},
        {"op": "eq", "args": [{"property": "data_format"}, "daily-average"]},
        {
            "op": "t_after",
            "args": [
                {"property": "time"},
                {"timestamp": "2023-02-01T00:00:00Z"},
            ],
        },
        {
            "op": "t_before",
            "args": [
                {"property": "time"},
                {"timestamp": "2023-02-28T23:59:59Z"},
            ],
        },
    ],
}

In [2]:
from pygeofilter_aeronet.evaluator import to_aeronet_api
query, _ = to_aeronet_api(cql2_filter)
query

2025-11-12 16:39:08.462 | DEBUG    | pygeofilter_aeronet:query_stations_from_parquet:174 - Executing query: SELECT * EXCLUDE(geometry), ST_AsWKB(geometry) as geometry FROM 'https://github.com/Terradue/pygeofilter-aeronet/raw/refs/heads/stations-update/stations.parquet'...


'site=Cart_Site&AOD10=1&if_no_html=1&AVG=20&year=2023&month=2&day=1&hour=0&year2=2023&month2=2&day2=28&hour2=23'

## Examples

### Level 1.0 data from the "Cart_Site" for AOD daily averages

Define a cql2-json filter for:

https://aeronet.gsfc.nasa.gov/cgi-bin/print_web_data_v3?site=Cart_Site&year=2000&month=6&day=1&year2=2000&month2=6&day2=14&AOD10=1&AVG=20



In [3]:
from folium import (
    GeoJson,
    LayerControl,
    Map
)
from folium.plugins import (
    Fullscreen
)
from IPython.display import (
    display,
    HTML
)
from geopandas import read_parquet
from pathlib import Path
from pygeofilter_aeronet import aeronet_search
from pystac import Item

out_dir: Path = Path('.')

cql2_filter = {
    "op": "and",
    "args": [
        {"op": "eq", "args": [{"property": "site"}, "Cart_Site"]},
        {"op": "eq", "args": [{"property": "data_type"}, "AOD10"]},
        {"op": "eq", "args": [{"property": "format"}, "csv"]},
        {"op": "eq", "args": [{"property": "data_format"}, "daily-average"]},
        {
            "op": "t_after",
            "args": [
                {"property": "time"},
                {"timestamp": "2000-06-01T00:00:00Z"},
            ],
        },
        {
            "op": "t_before",
            "args": [
                {"property": "time"},
                {"timestamp": "2000-06-14T23:59:59Z"},
            ],
        },
    ],
}

def render(cql2_filter) -> Map | None:
    item: Item | None = aeronet_search(
        cql2_filter=cql2_filter,
        output_dir=out_dir
    )

    if item:
        print(f"Item ID: {item.id}")
        geoparquet_file = item.get_assets()['geoparquet'].href
        print(f"GeoParquet output file: {geoparquet_file}")

        map = Map()
        layer_control = LayerControl(position="topright", collapsed=True)
        fullscreen = Fullscreen()
        style = {"fillColor": "#00000000", "color": "#0000ff", "weight": 1}

        footprints = GeoJson(
            read_parquet(geoparquet_file).to_json(),
            name="Stac Item footprints",
            style_function=lambda x: style,
            control=True,
        )

        footprints.add_to(map)
        layer_control.add_to(map)
        fullscreen.add_to(map)
        map.fit_bounds(map.get_bounds())
        return map

map: Map | None = render(cql2_filter)
map

2025-11-12 16:39:11.315 | SUCCESS  | pygeofilter_aeronet:aeronet_search:228 - Query on https://aeronet.gsfc.nasa.gov successfully obtained data:
2025-11-12 16:39:11.317 | SUCCESS  | pygeofilter_aeronet:aeronet_search:236 - Data saved to to CSV file: /home/stripodi/Documents/pygeofilter/pygeofilter-aeronet/docs/e3712370-88d8-41ea-9cc3-3fd8b6bc1086.csv
2025-11-12 16:39:11.327 | SUCCESS  | pygeofilter_aeronet:aeronet_search:247 - Data saved to GeoParquet file: /home/stripodi/Documents/pygeofilter/pygeofilter-aeronet/docs/e3712370-88d8-41ea-9cc3-3fd8b6bc1086.parquet


Item ID: urn:uuid:e3712370-88d8-41ea-9cc3-3fd8b6bc1086
GeoParquet output file: e3712370-88d8-41ea-9cc3-3fd8b6bc1086.parquet


## Level 1.5 data from all sites with available data for all AOD points

https://aeronet.gsfc.nasa.gov/cgi-bin/print_web_data_v3?year=2000&month=6&day=1&year2=2000&month2=6&day2=14&AOD15=1&AVG=10


In [ ]:
cql2_filter = {
    "op": "and",
    "args": [
        {"op": "eq", "args": [{"property": "data_type"}, "AOD15"]},
        {"op": "eq", "args": [{"property": "format"}, "csv"]},
        {"op": "eq", "args": [{"property": "data_format"}, "all-points"]},
        {
            "op": "t_after",
            "args": [
                {"property": "time"},
                {"timestamp": "2000-06-01T00:00:00Z"},
            ],
        },
        {
            "op": "t_before",
            "args": [
                {"property": "time"},
                {"timestamp": "2000-06-14T23:59:59Z"},
            ],
        },
    ],
}

map: Map | None = render(cql2_filter)
map


## Level 2.0 data from the "Cart_Site" for all AOD points

https://aeronet.gsfc.nasa.gov/cgi-bin/print_web_data_v3?site=Cart_Site&year=2000&month=6&day=1&year2=2000&month2=6&day2=14&AOD20=1&AVG=10


In [ ]:
cql2_filter = {
    "op": "and",
    "args": [
        {"op": "eq", "args": [{"property": "site"}, "Cart_Site"]},
        {"op": "eq", "args": [{"property": "data_type"}, "AOD20"]},
        {"op": "eq", "args": [{"property": "format"}, "csv"]},
        {"op": "eq", "args": [{"property": "data_format"}, "daily-average"]},
        {
            "op": "t_after",
            "args": [
                {"property": "time"},
                {"timestamp": "2000-06-01T00:00:00Z"},
            ],
        },
        {
            "op": "t_before",
            "args": [
                {"property": "time"},
                {"timestamp": "2000-06-14T23:59:59Z"},
            ],
        },
    ],
}

map: Map | None = render(cql2_filter)
map


## Level 2.0 data from the "Cart_Site" for all SDA points

https://aeronet.gsfc.nasa.gov/cgi-bin/print_web_data_v3?site=Cart_Site&year=2000&month=6&day=1&year2=2000&month2=6&day2=14&SDA20=1&AVG=10



In [ ]:
cql2_filter = {
    "op": "and",
    "args": [
        {"op": "eq", "args": [{"property": "site"}, "Cart_Site"]},
        {"op": "eq", "args": [{"property": "data_type"}, "SDA20"]},
        {"op": "eq", "args": [{"property": "format"}, "csv"]},
        {"op": "eq", "args": [{"property": "data_format"}, "daily-average"]},
        {
            "op": "t_after",
            "args": [
                {"property": "time"},
                {"timestamp": "2000-06-01T00:00:00Z"},
            ],
        },
        {
            "op": "t_before",
            "args": [
                {"property": "time"},
                {"timestamp": "2000-06-14T23:59:59Z"},
            ],
        },
    ],
}

map: Map | None = render(cql2_filter)
map

## Geographical search


In [ ]:
cql2_filter = {
    "op": "and",
    "args": [
        {"op": "eq", "args": [{"property": "data_type"}, "SDA20"]},
        {"op": "eq", "args": [{"property": "format"}, "csv"]},
        {"op": "eq", "args": [{"property": "data_format"}, "daily-average"]},
        {
            "op": "t_after",
            "args": [
                {"property": "time"},
                {"timestamp": "2000-06-01T00:00:00Z"},
            ],
        },
        {
            "op": "t_before",
            "args": [
                {"property": "time"},
                {"timestamp": "2000-06-14T23:59:59Z"},
            ],
        },
        {
            "op": "s_intersects",
            "args": [
                {"property": "geometry"},
                {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [8.0, 44.0],
                            [14.0, 44.0],
                            [14.0, 48.0],
                            [8.0, 48.0],
                            [8.0, 44.0],
                        ]
                    ],
                },
            ],
        },
    ],
}

map: Map | None = render(cql2_filter)
map